<a href="https://colab.research.google.com/github/ashishagg70/ContextBasedQA/blob/master/ContextBasedQAV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget "https://data.deepai.org/squad1.1.zip"

--2021-04-20 06:19:38--  https://data.deepai.org/squad1.1.zip
Resolving data.deepai.org (data.deepai.org)... 138.201.36.183
Connecting to data.deepai.org (data.deepai.org)|138.201.36.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9152254 (8.7M) [application/x-zip-compressed]
Saving to: ‘squad1.1.zip’

squad1.1.zip        100%[===================>]   8.73M  42.5MB/s    in 0.2s    

2021-04-20 06:19:38 (42.5 MB/s) - ‘squad1.1.zip’ saved [9152254/9152254]



In [2]:
!unzip squad1.1.zip

Archive:  squad1.1.zip
  inflating: dev-v1.1.json           
  inflating: train-v1.1.json         


In [93]:
! gdown --id 1JsjXsyXcYgAtOk6v3KijKOrAjJ_6LQpO #saved_model

Downloading...
From: https://drive.google.com/uc?id=1JsjXsyXcYgAtOk6v3KijKOrAjJ_6LQpO
To: /content/bidaf_model.tar.gz
2.42GB [00:31, 77.9MB/s]


In [3]:
import numpy as np
from collections import defaultdict
from nltk.tokenize import word_tokenize
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.vocab import GloVe
import json
import tqdm

import nltk
nltk.download('punkt')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
cuda


In [4]:
file = open('train-v1.1.json','r')
file2 = open('dev-v1.1.json','r')
train_data = json.load(file)['data']
dev_data = json.load(file2)['data']

In [5]:
MAX_WORD_LENGTH = 40
D = 100

In [6]:
glove = GloVe('840B',300)

.vector_cache/glove.840B.300d.zip: 2.18GB [07:20, 4.95MB/s]                            
100%|█████████▉| 2195891/2196017 [05:10<00:00, 7150.45it/s]

In [7]:
char_to_index = defaultdict(int)
index_to_char = defaultdict(str)
# word_to_index = defaultdict(int)
# index_to_word = defaultdict(str)

num_contexts = 0
num_questions = 0
num_characters = 0
num_words = len(glove)

char_contexts = []
char_questions = []

word_contexts=[]
word_questions = []
answers = []

input_data = []

In [8]:
# i=1
# for key in glove.itos:
#     word_to_index[key]=i
#     index_to_word[i]=key
#     i+=1

In [9]:
def char_encode(text, is_train=True):
    global char_to_index
    global index_to_char
    global num_characters

    text_endcoding = []
    for word in text:
        encoding = np.zeros(MAX_WORD_LENGTH)
        try:
            i=0
            for char in word:
                encode = char_to_index[char]
                if is_train == True and encode == 0:
                    index = num_characters+1
                    char_to_index[char]= index
                    index_to_char[index]= char
                    encode = index
                    num_characters+=1
                encoding[i]=encode
                i+=1
        except:
            print(word)
        text_endcoding.append(encoding)
    return text_endcoding

In [10]:
def convert_to_lower(text):
    return text.lower()

def perform_word_tokenization(text):
    return word_tokenize(text)

def word_encode(text, is_train=True):
    global word_to_index
    global index_to_word
    global num_words
    encoding =[]
    for word in text:
        try:
            encode = glove.stoi[word]
        except:
            encode = num_words
        encoding.append(encode)
    return encoding


In [11]:
def text_preprocess(text, is_train=True):
    text = convert_to_lower(text)
    text = perform_word_tokenization(text)
    word_encoding = word_encode(text)
    char_encoding = char_encode(text, is_train)
    return char_encoding, word_encoding

def get_answer_indices(context,start,answer):
    context=convert_to_lower(context)
    tokens = perform_word_tokenization(context[:(start+1)])
    ans_tokens = perform_word_tokenization(convert_to_lower(answer))
    start = len(tokens)-1
    return start, start + len(ans_tokens)-1

In [12]:
for obj in train_data:
    for para in obj['paragraphs']:
        cce, cwe = text_preprocess(para['context'])
        char_contexts.append(cce)
        word_contexts.append(cwe)
        num_contexts+=1
        for qa in para['qas']:
            qce, qwe = text_preprocess(qa['question'])
            char_questions.append(qce)
            word_questions.append(qwe)
            num_questions+=1
            input_data.append((num_contexts-1,num_questions-1))
            ans = []
            for a in qa['answers']:
                start,end = get_answer_indices(para['context'],a['answer_start'],a['text'])
                ans.append((start,end))
            answers.append(ans)

In [13]:
char_vocab_size = len(char_to_index)+1
word_vocab_size = num_words+1

In [14]:
class CharEmbedding(nn.Module):

    def __init__(self,vocab_size,embedding_dim = 8, cnn_kernel_size = 5,word_embedding_size =100 ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size,embedding_dim=embedding_dim,padding_idx=0,)
        self.cnn = nn.Conv1d(in_channels=embedding_dim,out_channels=word_embedding_size,kernel_size=cnn_kernel_size)
        self.maxpool = nn.MaxPool1d(kernel_size = MAX_WORD_LENGTH-cnn_kernel_size+1)
    
    def forward(self,x):
        # print("start",x.shape)
        batch_size = x.shape[0]
        x = x.view(-1,MAX_WORD_LENGTH)
        x = self.embedding(x)
        x = x.transpose(1,2)
        x = self.cnn(x)
        x = F.relu(x)
        # print("cnn",x.shape)
        x = self.maxpool(x)
        # print("pool",x.shape)
        #TODO : ReLU
        x = x.view(batch_size,-1,x.shape[1])
        # print("final",x.shape)
        return x

In [15]:
class WordEmbedding(nn.Module):
    def __init__(self):
        super().__init__()      
        self.embedding = nn.Embedding.from_pretrained(torch.cat((glove.vectors,torch.zeros(1,glove.dim)),dim=0))

    def forward(self,x):
        x=self.embedding(x)
        return x

In [16]:
class HighwayNetworkLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.transform = nn.Sequential(
               nn.Linear(glove.dim+D,glove.dim+D),
               nn.ReLU(),
               nn.Linear(glove.dim+D,glove.dim+D),
               nn.ReLU()
        )
        self.gate = nn.Sequential(
               nn.Linear(glove.dim+D,glove.dim+D),
               nn.Sigmoid()
        )
    def forward(self,x):
        x_transformed = self.transform(x)
        p = self.gate(x)
        return p*x_transformed + (1-p)*x

In [17]:
class ContextualEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.bilstm = nn.LSTM(glove.dim+D,D,1,bidirectional = True,batch_first = True)

    def forward(self,x):
        x,_ = self.bilstm(x)
        return x

In [18]:
class AttentionFlowLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.alpha = nn.Linear(6*D,1)
    
    def forward(self,H,U):
        # H : contextual embedding of context
        # U : contextual embedding of query

        T = H.shape[1]
        J = U.shape[1]

        H_interleaved = torch.repeat_interleave(H,J,dim=1)
        U_repeated = U.repeat(1,T,1)

        assert(H_interleaved.shape==U_repeated.shape)

        HU = torch.cat((H_interleaved,U_repeated,H_interleaved*U_repeated),dim=-1)
        S = self.alpha(HU)
        S = S.view(-1,T,J)
        C2Q_att = F.softmax(S,dim = -1)
        U_tilde = torch.matmul(C2Q_att,U)
        Q2C_att = F.softmax(torch.max(S,dim=-1)[0],dim=-1)
        Q2C_att = Q2C_att.unsqueeze(1)
        H_tilde = torch.matmul(Q2C_att,H).repeat(1,T,1)
        G = torch.cat((H,U_tilde,H*U_tilde,H*H_tilde),dim=-1)
        return G


In [19]:
class ModellingLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.bilstm = nn.LSTM(8*D,D,2,bidirectional = True,batch_first = True)
    def forward(self,G):
        M,_ = self.bilstm(G)
        return M

In [20]:
class OutputLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense1 = nn.Linear(10*D,1)
        self.dense2 = nn.Linear(10*D,1)
        self.bilstm = nn.LSTM(2*D,D,1,bidirectional = True,batch_first = True)
        self.softmax = nn.LogSoftmax(dim=-1)
    def forward(self,G,M):
        GM = torch.cat((G,M),dim=-1)
        temp_GM = self.dense1(GM).squeeze(-1)
        start = self.softmax(temp_GM)
        M2,_ = self.bilstm(M)
        GM2= torch.cat((G,M2),dim=-1)
        temp_GM2 = self.dense2(GM2).squeeze(-1)
        end = self.softmax(temp_GM2)

        return start,end

In [21]:
class BiDAF(nn.Module):
    def __init__(self):
        super().__init__()
        self.char_emb_layer=CharEmbedding(char_vocab_size,embedding_dim=8)
        self.word_emb_layer=WordEmbedding()
        self.highway = HighwayNetworkLayer()
        self.cont_emb_layer=ContextualEmbedding()
        self.att_layer = AttentionFlowLayer()
        self.modelling_layer = ModellingLayer()
        self.output_layer = OutputLayer()
    
    def forward(self,context_char,context_word,query_char,query_word):
        context_char_emb = self.char_emb_layer(context_char)
        context_word_emb = self.word_emb_layer(context_word)
        final_context_word_embedding = torch.cat((context_char_emb,context_word_emb),dim = -1)
        final_context_word_embedding = self.highway(final_context_word_embedding)
        context_cont_emb = self.cont_emb_layer(final_context_word_embedding)

        query_char_emb = self.char_emb_layer(query_char)
        query_word_emb = self.word_emb_layer(query_word)
        final_query_word_embedding = torch.cat((query_char_emb,query_word_emb),dim = -1)
        final_query_word_embedding = self.highway(final_query_word_embedding)
        query_cont_emb = self.cont_emb_layer(final_query_word_embedding)

        g = self.att_layer(context_cont_emb,query_cont_emb)
        m = self.modelling_layer(g)
        o = self.output_layer(g,m)
        return o

In [22]:
# char_batch_context = [char_contexts[0],char_contexts[0]]
# char_batch_query = [char_questions[0],char_questions[1]]
# word_batch_context=[word_contexts[0],word_contexts[0]]
# word_batch_query = [word_questions[0],word_questions[1]]

In [23]:
# char_batch_context_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in char_batch_context],batch_first=True))
# char_batch_query_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in char_batch_query],batch_first=True))
# word_batch_context_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in word_batch_context],batch_first=True))
# word_batch_query_t = torch.LongTensor(nn.utils.rnn.pad_sequence([torch.LongTensor(sent) for sent in word_batch_query],batch_first=True))

In [24]:
def collate_fn(idx):
    context_char,context_word,query_char,query_word,ans_start, ans_end=[],[],[],[],[],[]
    for id in idx:
        c,q = input_data[id]
        context_char.append(torch.LongTensor(char_contexts[c]))
        context_word.append(torch.LongTensor(word_contexts[c]))
        query_char.append(torch.LongTensor(char_questions[q]))
        query_word.append(torch.LongTensor(word_questions[q]))
        start,end = answers[q][0]
        ans_start.append(start)
        ans_end.append(end)
    
    context_char = torch.LongTensor(nn.utils.rnn.pad_sequence(context_char,batch_first=True))
    context_word = torch.LongTensor(nn.utils.rnn.pad_sequence(context_word,batch_first=True))
    query_char = torch.LongTensor(nn.utils.rnn.pad_sequence(query_char,batch_first=True))
    query_word = torch.LongTensor(nn.utils.rnn.pad_sequence(query_word,batch_first=True))
    ans_start = torch.LongTensor(ans_start)
    ans_end = torch.LongTensor(ans_end)

    return context_char,context_word,query_char,query_word,ans_start,ans_end,idx

In [25]:
batch_size = 32
# num_workers = 1

In [26]:
train_loader = torch.utils.data.DataLoader(
    range(len(input_data)),
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    # num_workers=num_workers,
    # pin_memory=pin_memory
)

In [27]:
bidaf = BiDAF().to(device)

In [28]:
epochs = 12

learning_rate = 0.5

print_every = 50

optimizer = torch.optim.Adadelta(bidaf.parameters(),lr=learning_rate)

criterion = nn.NLLLoss()


In [29]:
bidaf.train()

for epoch in range(epochs):
    for batch_idx, _data in enumerate(tqdm.notebook.tqdm(iter(train_loader))):
        context_char,context_word,query_char,query_word,ans_start,ans_end,idx = _data
        context_char,context_word,query_char,query_word,ans_start,ans_end = context_char.to(device),context_word.to(device),query_char.to(device),query_word.to(device),ans_start.to(device),ans_end.to(device)
        optimizer.zero_grad()
        start,end = bidaf(context_char,context_word,query_char,query_word)
        loss1 = criterion(start,ans_start)
        loss2 = criterion(end,ans_end)
        loss = loss1+loss2
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            if batch_idx % print_every == 0 :
                print("epoch = {},batch = {},loss={},loss_1={},loss_2={}".format(epoch,batch_idx,loss.item(),loss1.item(),loss2.item()))
        

epoch = 0,batch = 0,loss=11.312808990478516,loss_1=5.640651702880859,loss_2=5.672156810760498
epoch = 0,batch = 50,loss=8.233719825744629,loss_1=4.2565717697143555,loss_2=3.9771478176116943
epoch = 0,batch = 100,loss=8.067789077758789,loss_1=4.118606090545654,loss_2=3.949183225631714
epoch = 0,batch = 150,loss=7.60382604598999,loss_1=3.9206295013427734,loss_2=3.683196544647217
epoch = 0,batch = 200,loss=7.588215351104736,loss_1=3.7870798110961914,loss_2=3.801135540008545
epoch = 0,batch = 250,loss=7.7616801261901855,loss_1=4.134261131286621,loss_2=3.6274189949035645
epoch = 0,batch = 300,loss=7.748405456542969,loss_1=4.164091110229492,loss_2=3.5843141078948975
epoch = 0,batch = 350,loss=7.891063690185547,loss_1=4.106838703155518,loss_2=3.7842252254486084
epoch = 0,batch = 400,loss=7.958622932434082,loss_1=4.04783821105957,loss_2=3.9107844829559326
epoch = 0,batch = 450,loss=7.421836853027344,loss_1=3.967987537384033,loss_2=3.4538495540618896
epoch = 0,batch = 500,loss=8.172725677490234

epoch = 1,batch = 0,loss=4.337512969970703,loss_1=2.290494441986084,loss_2=2.0470187664031982
epoch = 1,batch = 50,loss=4.784224510192871,loss_1=2.5811891555786133,loss_2=2.2030351161956787
epoch = 1,batch = 100,loss=4.427314758300781,loss_1=2.2717275619506836,loss_2=2.1555871963500977
epoch = 1,batch = 150,loss=5.5632171630859375,loss_1=2.6189372539520264,loss_2=2.944279670715332
epoch = 1,batch = 200,loss=5.5848388671875,loss_1=2.7601611614227295,loss_2=2.8246774673461914
epoch = 1,batch = 250,loss=4.700056076049805,loss_1=2.4260976314544678,loss_2=2.273958683013916
epoch = 1,batch = 300,loss=4.887452125549316,loss_1=2.3796639442443848,loss_2=2.5077884197235107
epoch = 1,batch = 350,loss=4.792760372161865,loss_1=2.562251567840576,loss_2=2.230508804321289
epoch = 1,batch = 400,loss=3.718167304992676,loss_1=1.8693819046020508,loss_2=1.848785400390625
epoch = 1,batch = 450,loss=4.291177749633789,loss_1=2.3627591133117676,loss_2=1.9284186363220215
epoch = 1,batch = 500,loss=4.41645240783

epoch = 2,batch = 0,loss=3.717777729034424,loss_1=2.1087331771850586,loss_2=1.6090446710586548
epoch = 2,batch = 50,loss=3.080641269683838,loss_1=1.5597646236419678,loss_2=1.5208767652511597
epoch = 2,batch = 100,loss=3.4093594551086426,loss_1=1.6985194683074951,loss_2=1.710839867591858
epoch = 2,batch = 150,loss=3.102768659591675,loss_1=1.6656250953674316,loss_2=1.4371435642242432
epoch = 2,batch = 200,loss=3.7288706302642822,loss_1=1.9424622058868408,loss_2=1.7864084243774414
epoch = 2,batch = 250,loss=2.684349536895752,loss_1=1.3822367191314697,loss_2=1.3021126985549927
epoch = 2,batch = 300,loss=2.343212127685547,loss_1=1.2426918745040894,loss_2=1.1005202531814575
epoch = 2,batch = 350,loss=3.6414923667907715,loss_1=1.983563780784607,loss_2=1.6579285860061646
epoch = 2,batch = 400,loss=3.371817111968994,loss_1=1.7145036458969116,loss_2=1.657313585281372
epoch = 2,batch = 450,loss=3.6234869956970215,loss_1=1.7004119157791138,loss_2=1.9230750799179077
epoch = 2,batch = 500,loss=2.851

epoch = 3,batch = 0,loss=2.522751569747925,loss_1=1.3382327556610107,loss_2=1.184518814086914
epoch = 3,batch = 50,loss=2.699465274810791,loss_1=1.4373811483383179,loss_2=1.2620842456817627
epoch = 3,batch = 100,loss=2.853785991668701,loss_1=1.2951197624206543,loss_2=1.5586663484573364
epoch = 3,batch = 150,loss=2.293018341064453,loss_1=1.3721485137939453,loss_2=0.9208699464797974
epoch = 3,batch = 200,loss=2.1445703506469727,loss_1=1.0782853364944458,loss_2=1.0662851333618164
epoch = 3,batch = 250,loss=2.5571980476379395,loss_1=1.2198882102966309,loss_2=1.3373098373413086
epoch = 3,batch = 300,loss=2.9710841178894043,loss_1=1.6426753997802734,loss_2=1.3284087181091309
epoch = 3,batch = 350,loss=2.8838109970092773,loss_1=1.5220662355422974,loss_2=1.3617448806762695
epoch = 3,batch = 400,loss=2.241273880004883,loss_1=1.2464686632156372,loss_2=0.9948053359985352
epoch = 3,batch = 450,loss=3.122284412384033,loss_1=1.6223829984664917,loss_2=1.4999014139175415
epoch = 3,batch = 500,loss=1.8

epoch = 4,batch = 0,loss=2.8239564895629883,loss_1=1.4445405006408691,loss_2=1.3794161081314087
epoch = 4,batch = 50,loss=1.3031554222106934,loss_1=0.8032797574996948,loss_2=0.4998757243156433
epoch = 4,batch = 100,loss=2.7080914974212646,loss_1=1.540053129196167,loss_2=1.1680383682250977
epoch = 4,batch = 150,loss=1.9335095882415771,loss_1=0.7798779606819153,loss_2=1.153631567955017
epoch = 4,batch = 200,loss=2.0352468490600586,loss_1=1.0090022087097168,loss_2=1.0262445211410522
epoch = 4,batch = 250,loss=2.361093282699585,loss_1=1.3389146327972412,loss_2=1.0221786499023438
epoch = 4,batch = 300,loss=2.4186925888061523,loss_1=1.2951377630233765,loss_2=1.1235549449920654
epoch = 4,batch = 350,loss=1.8111045360565186,loss_1=0.8869596719741821,loss_2=0.9241449236869812
epoch = 4,batch = 400,loss=2.1160340309143066,loss_1=1.1902071237564087,loss_2=0.9258267879486084
epoch = 4,batch = 450,loss=1.9441403150558472,loss_1=1.0081071853637695,loss_2=0.9360331296920776
epoch = 4,batch = 500,loss

...

epoch = 7,batch = 0,loss=1.747135877609253,loss_1=0.8948209285736084,loss_2=0.8523149490356445
epoch = 7,batch = 50,loss=1.5429455041885376,loss_1=0.8345303535461426,loss_2=0.708415150642395
epoch = 7,batch = 100,loss=1.6691648960113525,loss_1=0.9764727354049683,loss_2=0.6926921606063843
epoch = 7,batch = 150,loss=1.8674752712249756,loss_1=1.2811996936798096,loss_2=0.5862755179405212
epoch = 7,batch = 200,loss=2.4730236530303955,loss_1=1.382902979850769,loss_2=1.0901206731796265
epoch = 7,batch = 250,loss=1.109337329864502,loss_1=0.6626977324485779,loss_2=0.4466395974159241
epoch = 7,batch = 300,loss=0.9904830455780029,loss_1=0.5796077847480774,loss_2=0.41087523102760315
epoch = 7,batch = 350,loss=1.5556614398956299,loss_1=0.5852454900741577,loss_2=0.9704160094261169
epoch = 7,batch = 400,loss=1.019789457321167,loss_1=0.5360372066497803,loss_2=0.4837522506713867
epoch = 7,batch = 450,loss=2.235043525695801,loss_1=1.2379058599472046,loss_2=0.9971375465393066
epoch = 7,batch = 500,loss=1

epoch = 8,batch = 0,loss=1.395193338394165,loss_1=0.6117938756942749,loss_2=0.7833995223045349
epoch = 8,batch = 50,loss=1.5631177425384521,loss_1=0.7927046418190002,loss_2=0.7704130411148071
epoch = 8,batch = 100,loss=1.205986499786377,loss_1=0.7687774300575256,loss_2=0.4372090697288513
epoch = 8,batch = 150,loss=1.395704746246338,loss_1=0.694234311580658,loss_2=0.7014703750610352
epoch = 8,batch = 200,loss=1.0168521404266357,loss_1=0.3881702423095703,loss_2=0.6286819577217102
epoch = 8,batch = 250,loss=0.8207574486732483,loss_1=0.412263423204422,loss_2=0.4084940254688263
epoch = 8,batch = 300,loss=1.0100759267807007,loss_1=0.501338541507721,loss_2=0.5087373852729797
epoch = 8,batch = 350,loss=0.9275602102279663,loss_1=0.4937525987625122,loss_2=0.4338075816631317
epoch = 8,batch = 400,loss=1.1262376308441162,loss_1=0.6165699362754822,loss_2=0.5096676349639893
epoch = 8,batch = 450,loss=1.5416009426116943,loss_1=0.7985668182373047,loss_2=0.7430341839790344
epoch = 8,batch = 500,loss=1.

epoch = 9,batch = 0,loss=1.1082358360290527,loss_1=0.606471061706543,loss_2=0.5017647743225098
epoch = 9,batch = 50,loss=0.7951450943946838,loss_1=0.42868107557296753,loss_2=0.3664640188217163
epoch = 9,batch = 100,loss=0.9335876107215881,loss_1=0.5669349431991577,loss_2=0.3666526675224304
epoch = 9,batch = 150,loss=1.2472679615020752,loss_1=0.561757504940033,loss_2=0.6855103969573975
epoch = 9,batch = 200,loss=0.6436735391616821,loss_1=0.4459238350391388,loss_2=0.19774971902370453
epoch = 9,batch = 250,loss=1.343360185623169,loss_1=0.7962827086448669,loss_2=0.5470775365829468
epoch = 9,batch = 300,loss=0.5182980298995972,loss_1=0.2990671694278717,loss_2=0.21923086047172546
epoch = 9,batch = 350,loss=1.1544761657714844,loss_1=0.5635656118392944,loss_2=0.5909104943275452
epoch = 9,batch = 400,loss=0.982974112033844,loss_1=0.6531771421432495,loss_2=0.3297969698905945
epoch = 9,batch = 450,loss=0.8649416565895081,loss_1=0.5344406366348267,loss_2=0.3305010199546814
epoch = 9,batch = 500,lo

epoch = 10,batch = 0,loss=1.0046520233154297,loss_1=0.5025896430015564,loss_2=0.5020623803138733
epoch = 10,batch = 50,loss=0.9259853363037109,loss_1=0.6377865672111511,loss_2=0.2881987690925598
epoch = 10,batch = 100,loss=0.7728575468063354,loss_1=0.370640367269516,loss_2=0.40221720933914185
epoch = 10,batch = 150,loss=1.3869551420211792,loss_1=0.6495012640953064,loss_2=0.7374538779258728
epoch = 10,batch = 200,loss=0.9820715188980103,loss_1=0.483280748128891,loss_2=0.4987907409667969
epoch = 10,batch = 250,loss=0.7649130821228027,loss_1=0.39948979020118713,loss_2=0.3654232919216156
epoch = 10,batch = 300,loss=1.1662935018539429,loss_1=0.5755394101142883,loss_2=0.5907540917396545
epoch = 10,batch = 350,loss=0.650991678237915,loss_1=0.3273031711578369,loss_2=0.3236885070800781
epoch = 10,batch = 400,loss=0.8166605234146118,loss_1=0.36564967036247253,loss_2=0.4510108232498169
epoch = 10,batch = 450,loss=1.0732219219207764,loss_1=0.6177789568901062,loss_2=0.4554429054260254
epoch = 10,ba

epoch = 11,batch = 0,loss=0.7449941635131836,loss_1=0.44230926036834717,loss_2=0.3026849031448364
epoch = 11,batch = 50,loss=0.8827968239784241,loss_1=0.5088399052619934,loss_2=0.37395691871643066
epoch = 11,batch = 100,loss=0.5642915964126587,loss_1=0.31896349787712097,loss_2=0.2453281283378601
epoch = 11,batch = 150,loss=0.9307068586349487,loss_1=0.5506441593170166,loss_2=0.3800627291202545
epoch = 11,batch = 200,loss=1.0195995569229126,loss_1=0.5711931586265564,loss_2=0.4484063684940338
epoch = 11,batch = 250,loss=0.579866886138916,loss_1=0.30459800362586975,loss_2=0.27526891231536865
epoch = 11,batch = 300,loss=1.5050148963928223,loss_1=0.7261644601821899,loss_2=0.7788503766059875
epoch = 11,batch = 350,loss=0.7762411236763,loss_1=0.33720022439956665,loss_2=0.4390408992767334
epoch = 11,batch = 400,loss=0.5856414437294006,loss_1=0.30597594380378723,loss_2=0.2796654999256134
epoch = 11,batch = 450,loss=0.6205910444259644,loss_1=0.3648315668106079,loss_2=0.25575947761535645
epoch = 1

In [72]:
torch.save(bidaf, "bidaf_model")

In [75]:
!ls -lah

total 2.6G
drwxr-xr-x 1 root root 4.0K Apr 20 12:26 .
drwxr-xr-x 1 root root 4.0K Apr 20 06:18 ..
-rw-r--r-- 1 root root 2.5G Apr 20 12:26 bidaf_model
drwxr-xr-x 4 root root 4.0K Apr  7 13:35 .config
-rw-r--r-- 1 root root 4.7M Nov 17  2019 dev-v1.1.json
drwxr-xr-x 1 root root 4.0K Apr  7 13:36 sample_data
-rw-r--r-- 1 root root 8.8M Jul 14  2020 squad1.1.zip
-rw-r--r-- 1 root root  29M Nov 17  2019 train-v1.1.json
drwxr-xr-x 2 root root 4.0K Apr 20 06:34 .vector_cache


In [35]:
train_data[0]

{'paragraphs': [{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
   'qas': [{'answers': [{'answer_start': 515,
       'text': 'Saint Bernadette Soubirous'}],
     'id': '5733be284776f41900661182',
     'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'},
    {'answers': [{'answer_start': 188, 'text': 

In [64]:
context_char_test,context_word_test,query_char_test,query_word_test,ans_start_test, ans_end_test=[],[],[],[],[],[]
c,q = input_data[3]
context_char_test.append(torch.LongTensor(char_contexts[c]))
context_word_test.append(torch.LongTensor(word_contexts[c]))
query_char_test.append(torch.LongTensor(char_questions[q]))
query_word_test.append(torch.LongTensor(word_questions[q]))
start_test,end_test = answers[q][0]
ans_start_test.append(start_test)
ans_end_test.append(end_test)
context_char_test = torch.LongTensor(nn.utils.rnn.pad_sequence(context_char_test,batch_first=True))
context_word_test = torch.LongTensor(nn.utils.rnn.pad_sequence(context_word_test,batch_first=True))
query_char_test = torch.LongTensor(nn.utils.rnn.pad_sequence(query_char_test,batch_first=True))
query_word_test = torch.LongTensor(nn.utils.rnn.pad_sequence(query_word_test,batch_first=True))
context_char_test,context_word_test,query_char_test,query_word_test= context_char_test.to(device),context_word_test.to(device),query_char_test.to(device),query_word_test.to(device)

In [65]:
start,end = bidaf(context_char_test,context_word_test,query_char_test,query_word_test)
torch.argmax(start),torch.argmax(end)

(tensor(76, device='cuda:0'), tensor(82, device='cuda:0'))

In [66]:
ans_start_test, ans_end_test

([76], [82])